In [8]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import floor, ceil, sqrt
from sklearn.metrics import auc

############### PARAMS ##################
tag = "sbmal_3_balanced_val[0]_v4"
show_selector = lambda x: True
#########################################

wandb.login()

"""
https://stackoverflow.com/questions/21654635/scatter-plots-in-pandas-pyplot-how-to-plot-by-category
"""

def afterPoint(str):
    return str.split('.')[-1]
    
def getSelectorName(run):
    active_learning_selector = afterPoint(run.config["experiment/active_learning_selector"])
    if active_learning_selector == "UNCERTAINTY" or active_learning_selector == "UNCERTAINTY_INVERTED":
        uncertainty_name = "ALEA" if afterPoint(run.config['experiment/active_learning_selector_uncertainty_mode']) == 'ALEATORIC' else 'EPIST'
        prop_name = 'ISO' if afterPoint(run.config['experiment/active_learning_selector_network_mode']) == 'ISOLATED' else 'PROP'
        active_learning_selector = f"{'-' if active_learning_selector == 'UNCERTAINTY_INVERTED' else ''}{uncertainty_name} {prop_name}"
    elif active_learning_selector == "FIXED":
        prop_name = run.config['experiment/active_learning_training_type']
        active_learning_selector = f"{active_learning_selector} {prop_name}"
        
    return active_learning_selector

SUMMARY_FINAL_ACCURACY = '/eval/VALTEST/PROPAGATED/accuracy'
AL_PREFIX = 'mean/al/'
AL_SUFFIX = '/VALTEST/PROPAGATED/accuracy'

api = wandb.Api(timeout=100)
runs = api.runs("tum_daml_ba_antoniooroz/GR2", {"tags" : tag})

runs_per_model = {}

values_per_model_selector = {}

for run in runs:
    if run.config["debug"] or run.state != 'finished':
        continue
    
    model_type = str(run.config["model/type"])
    
    if model_type in runs_per_model:
        runs_per_model[model_type].append(run)
    else:
        runs_per_model[model_type] = [run]
        
for key, runs in runs_per_model.items():
    datapoints = []
    SAMPLES = (run.config['experiment/seeds/end'] - run.config['experiment/seeds/start']) * run.config['experiment/iterations_per_seed']
    
    for run in runs:
        model = afterPoint(run.config["model/type"])
        cycles = ceil(run.config['experiment/active_learning_budget']/run.config['experiment/active_learning_budget_per_update'])
        active_learning_selector = getSelectorName(run)

        for summary_key, summary_value in run.summary.items():
            if summary_key.startswith(AL_PREFIX) and summary_key.endswith(AL_SUFFIX):
                num = int(summary_key.replace(AL_PREFIX, '').replace(AL_SUFFIX, ''))
                
                if num == 0:
                    continue
                
                if f"{model} {active_learning_selector}" not in values_per_model_selector:
                    values_per_model_selector[f"{model} {active_learning_selector}"] = [None] * cycles
                
                values_per_model_selector[f"{model} {active_learning_selector}"][num-1]=summary_value
                
        values_per_model_selector[f"{model} {active_learning_selector}"][cycles-1]=run.summary[f"mean{SUMMARY_FINAL_ACCURACY}"]
        
#print(values_per_model_selector)

x = np.arange(0,1.01,1/(cycles-1))

for key, values in values_per_model_selector.items():
    print(f"{key}: {auc(x, np.array(values))}")


GPN FIXED [0, 1, 2, 3, 4, 5]: 0.3615887738191164
GPN FIXED [5]: 0.23261047383913627
GPN FIXED [4]: 0.31372085729470617
GPN EPIST PROP: 0.1998717228953655
GPN EPIST ISO: 0.22223352640867233
GPN ALEA ISO: 0.2071402709071453
GPN RANDOM: 0.19185839879971284
GPN L2_DISTANCE: 0.23359675533496416
GPN FIXED [1]: 0.4156448032993537
GPN FIXED [3]: 0.40070731422075856
GPN FIXED [2]: 0.40632989716071355
GPN FIXED [0]: 0.47464467642398983
MCD_GCN EPIST ISO: 0.5158978391152163
MCD_GCN FIXED [0, 1, 2, 3, 4, 5]: 0.56308079797488
MCD_GCN FIXED [5]: 0.3510572428886707
MCD_GCN FIXED [4]: 0.5327759236097336
MCD_GCN FIXED [3]: 0.5956707768715346
MCD_GCN FIXED [2]: 0.6859726160764693
MCD_GCN FIXED [1]: 0.7253423367555325
MCD_GCN FIXED [0]: 0.7618090510368346
MCD_GCN EPIST PROP: 0.5449946511250275
MCD_GCN ALEA PROP: 0.4154108132307346
MCD_GCN ALEA ISO: 0.5468035523708049
MCD_GCN RANDOM: 0.5371699665601437
MCD_GCN L2_DISTANCE: 0.5231642826245381
MCD_PPNP EPIST ISO: 0.6177816585852549
MCD_PPNP ALEA PROP: 0.612